In [1]:
from pyperplan import _parse

# LOADING A PROBLEM

In [2]:
problem = _parse(domain_file='testdomain.pddl',problem_file='testinstance.pddl')

In [3]:
problem.initial_state

[adjacent[('loc1', location), ('loc2', location)],
 adjacent[('loc2', location), ('loc1', location)],
 in[('conta', container), ('loc1', location)],
 in[('contb', container), ('loc2', location)],
 atl[('robr', robot), ('loc1', location)],
 atl[('robq', robot), ('loc2', location)],
 unloaded[('robr', robot)],
 unloaded[('robq', robot)]]

In [4]:
problem.name

'dwrpb1'

In [5]:
problem.objects

{'robr': robot,
 'robq': robot,
 'loc1': location,
 'loc2': location,
 'conta': container,
 'contb': container}

In [6]:
problem.goal

[in[('contb', (container,)), ('loc1', (location,))],
 loaded[('robr', (robot,)), ('conta', (container,))]]

In [7]:
[print(pred) for pred in problem.domain.predicates]

adjacent
atl
loaded
unloaded
in


[None, None, None, None, None]

# GROUNDING(?) IT

In [8]:
from pyperplan import _ground

In [9]:
task = _ground(problem)

In [10]:
task

<Task dwrpb1, vars: 14, operators: 20>

In [11]:
task.facts

{'(atl robq loc1)',
 '(atl robq loc2)',
 '(atl robr loc1)',
 '(atl robr loc2)',
 '(in conta loc1)',
 '(in conta loc2)',
 '(in contb loc1)',
 '(in contb loc2)',
 '(loaded robq conta)',
 '(loaded robq contb)',
 '(loaded robr conta)',
 '(loaded robr contb)',
 '(unloaded robq)',
 '(unloaded robr)'}

In [16]:
task.initial_state

frozenset({'(atl robq loc2)',
           '(atl robr loc1)',
           '(in conta loc1)',
           '(in contb loc2)',
           '(unloaded robq)',
           '(unloaded robr)'})

In [13]:
task.goals

frozenset({'(in contb loc1)', '(loaded robr conta)'})

In [14]:
task.goal_reached(task.initial_state)

False

In [23]:
task.get_successor_states(frozenset(task.initial_state))

[(<Op (move robr loc1 loc2)>,
  frozenset({'(atl robq loc2)',
             '(atl robr loc2)',
             '(in conta loc1)',
             '(in contb loc2)',
             '(unloaded robq)',
             '(unloaded robr)'})),
 (<Op (move robq loc2 loc1)>,
  frozenset({'(atl robq loc1)',
             '(atl robr loc1)',
             '(in conta loc1)',
             '(in contb loc2)',
             '(unloaded robq)',
             '(unloaded robr)'})),
 (<Op (load loc2 contb robq)>,
  frozenset({'(atl robq loc2)',
             '(atl robr loc1)',
             '(in conta loc1)',
             '(loaded robq contb)',
             '(unloaded robr)'})),
 (<Op (load loc1 conta robr)>,
  frozenset({'(atl robq loc2)',
             '(atl robr loc1)',
             '(in contb loc2)',
             '(loaded robr conta)',
             '(unloaded robq)'}))]

In [15]:
task.operators

[<Op (move robq loc1 loc2)>,
 <Op (move robq loc2 loc1)>,
 <Op (move robr loc1 loc2)>,
 <Op (move robr loc2 loc1)>,
 <Op (load loc1 conta robq)>,
 <Op (load loc1 conta robr)>,
 <Op (load loc1 contb robq)>,
 <Op (load loc1 contb robr)>,
 <Op (load loc2 conta robq)>,
 <Op (load loc2 conta robr)>,
 <Op (load loc2 contb robq)>,
 <Op (load loc2 contb robr)>,
 <Op (unload loc1 conta robq)>,
 <Op (unload loc1 conta robr)>,
 <Op (unload loc1 contb robq)>,
 <Op (unload loc1 contb robr)>,
 <Op (unload loc2 conta robq)>,
 <Op (unload loc2 conta robr)>,
 <Op (unload loc2 contb robq)>,
 <Op (unload loc2 contb robr)>]

# HEURISTIC

In [16]:
from pyperplan import get_heuristics, _get_heuristic_name

In [17]:
HEURISTICS = {_get_heuristic_name(heur): heur for heur in get_heuristics()}

In [18]:
# These are just heuristic classes
HEURISTICS

{'blind': blind.BlindHeuristic,
 'hadd': relaxation.hAddHeuristic,
 'hff': relaxation.hFFHeuristic,
 'hmax': relaxation.hMaxHeuristic,
 'hsa': relaxation.hSAHeuristic,
 'lmcut': lm_cut.LmCutHeuristic,
 'landmark': landmarks.LandmarkHeuristic}

In [19]:
# Defines the heuristic for this task? 
h = HEURISTICS['hmax'](task)

In [20]:
h

# SEARCH

In [21]:
from pyperplan import search

In [22]:
SEARCHES = {
    'astar': search.astar_search,
    'wastar': search.weighted_astar_search,
    'gbf': search.greedy_best_first_search,
    'bfs': search.breadth_first_search,
    'ehs': search.enforced_hillclimbing_search,
    'ids': search.iterative_deepening_search,
    'sat': search.sat_solve,
}

In [24]:
solution = SEARCHES['astar'](task, h)

In [25]:
solution

[<Op (load loc2 contb robq)>,
 <Op (move robq loc2 loc1)>,
 <Op (load loc1 conta robr)>,
 <Op (unload loc1 contb robq)>]

In [26]:
[op.name for op in solution]

['(load loc2 contb robq)',
 '(move robq loc2 loc1)',
 '(load loc1 conta robr)',
 '(unload loc1 contb robq)']

# VALIDATE SOLUTION

In [49]:
from pyperplan import _write_solution
import subprocess

In [50]:
_write_solution(solution,'testsolution.txt')

In [51]:
exitcode = subprocess.call(['./validate', 'testdomain.pddl','testinstance.pddl','testsolution.txt'], stdout=subprocess.PIPE)

In [52]:
if exitcode == 0:
    print('Plan correct!')
else:
    print('Plan NOT correct!')

Plan correct!
